In [163]:
import pandas as pd
import featuretools as ft
import numpy as np
import composeml as cp
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [98]:
ben_data = pd.read_csv("./HealthCare/Train_Beneficiarydata.csv").sample(frac=0.5)
admit_patient_data = pd.read_csv("./HealthCare/Train_Inpatientdata.csv").sample(frac=0.5)
checked_patient_data = pd.read_csv("./HealthCare/Train_Outpatientdata.csv").sample(frac=0.5)
labels = pd.read_csv("./HealthCare/Train.csv")

In [99]:
ben_data.head(5)

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
62601,BENE78023,1933-11-01,NaN,2,1,0,33,650,12,12,...,1,1,1,2,2,2,9000,1068,2880,880
69734,BENE85657,1931-12-01,NaN,1,2,0,42,30,12,12,...,2,2,2,2,2,2,0,0,110,0
31078,BENE44310,1942-10-01,NaN,1,1,0,10,260,12,12,...,2,2,2,2,2,2,0,0,600,0
135792,BENE156228,1934-12-01,NaN,1,1,Y,10,350,12,12,...,2,1,2,1,2,2,2000,1068,860,200
100979,BENE119025,1915-10-01,NaN,1,1,0,52,40,12,12,...,2,1,1,2,2,2,0,0,730,600


In [100]:
admit_patient_data.head(5)

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
17161,BENE74028,CLM80776,2009-12-26,2009-12-28,PRV55462,18000,PHY422134,PHY429430,NaN,2009-12-26,...,NaN,NaN,NaN,NaN,3971.0,NaN,NaN,NaN,NaN,NaN
34759,BENE138791,CLM45805,2009-04-06,2009-04-10,PRV52999,9000,PHY392852,NaN,NaN,2009-04-06,...,72252,1622,2761,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13457,BENE60585,CLM32859,2009-01-07,2009-01-07,PRV56560,19000,PHY352642,PHY384128,NaN,2009-01-07,...,V103,5859,NaN,NaN,66.0,2724.0,NaN,NaN,NaN,NaN
1131,BENE14931,CLM73818,2009-10-29,2009-10-30,PRV52831,8000,PHY372867,NaN,NaN,2009-10-29,...,99681,5641,2948,2930,NaN,NaN,NaN,NaN,NaN,NaN
34785,BENE138894,CLM61293,2009-07-26,2009-07-28,PRV52065,3000,PHY389703,NaN,NaN,2009-07-26,...,2724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
checked_patient_data.head(5)

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
61127,BENE28435,CLM751605,2009-12-30,2009-12-30,PRV51164,30,PHY432396,NaN,NaN,V734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
53885,BENE26307,CLM640390,2009-10-20,2009-10-20,PRV56416,800,PHY342652,NaN,PHY355570,5738,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
368736,BENE116615,CLM718712,2009-12-07,2009-12-07,PRV52045,40,PHY424585,NaN,NaN,53020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
403744,BENE126615,CLM569499,2009-09-08,2009-09-08,PRV51929,500,PHY378288,NaN,PHY393147,72081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
106974,BENE41584,CLM397918,2009-06-05,2009-06-05,PRV53918,600,PHY330576,NaN,NaN,9926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [102]:
labels.head(5)

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


## Preprocessing

In [103]:
ben_data.isna().sum()

BeneID                                 0
DOB                                    0
DOD                                68598
Gender                                 0
Race                                   0
RenalDiseaseIndicator                  0
State                                  0
County                                 0
NoOfMonths_PartACov                    0
NoOfMonths_PartBCov                    0
ChronicCond_Alzheimer                  0
ChronicCond_Heartfailure               0
ChronicCond_KidneyDisease              0
ChronicCond_Cancer                     0
ChronicCond_ObstrPulmonary             0
ChronicCond_Depression                 0
ChronicCond_Diabetes                   0
ChronicCond_IschemicHeart              0
ChronicCond_Osteoporasis               0
ChronicCond_rheumatoidarthritis        0
ChronicCond_stroke                     0
IPAnnualReimbursementAmt               0
IPAnnualDeductibleAmt                  0
OPAnnualReimbursementAmt               0
OPAnnualDeductib

In [104]:
## Amost all cells contains not null data

In [105]:
print(len(admit_patient_data))
admit_patient_data.isna().sum()

20237


BeneID                        0
ClaimID                       0
ClaimStartDt                  0
ClaimEndDt                    0
Provider                      0
InscClaimAmtReimbursed        0
AttendingPhysician           58
OperatingPhysician         8391
OtherPhysician            17873
AdmissionDt                   0
ClmAdmitDiagnosisCode         0
DeductibleAmtPaid           465
DischargeDt                   0
DiagnosisGroupCode            0
ClmDiagnosisCode_1            0
ClmDiagnosisCode_2          102
ClmDiagnosisCode_3          341
ClmDiagnosisCode_4          785
ClmDiagnosisCode_5         1469
ClmDiagnosisCode_6         2433
ClmDiagnosisCode_7         3653
ClmDiagnosisCode_8         5001
ClmDiagnosisCode_9         6813
ClmDiagnosisCode_10       18282
ClmProcedureCode_1         8719
ClmProcedureCode_2        17556
ClmProcedureCode_3        19758
ClmProcedureCode_4        20177
ClmProcedureCode_5        20231
ClmProcedureCode_6        20237
dtype: int64

In [106]:
### Only 9 patients went for proc 5 and none for 6. Drop these.

In [107]:
print(len(checked_patient_data))
checked_patient_data.isna().sum()

258868


BeneID                         0
ClaimID                        0
ClaimStartDt                   0
ClaimEndDt                     0
Provider                       0
InscClaimAmtReimbursed         0
AttendingPhysician           711
OperatingPhysician        213407
OtherPhysician            161090
ClmDiagnosisCode_1          5116
ClmDiagnosisCode_2         97529
ClmDiagnosisCode_3        157219
ClmDiagnosisCode_4        196084
ClmDiagnosisCode_5        221659
ClmDiagnosisCode_6        234268
ClmDiagnosisCode_7        242210
ClmDiagnosisCode_8        247260
ClmDiagnosisCode_9        251336
ClmDiagnosisCode_10       258322
ClmProcedureCode_1        258785
ClmProcedureCode_2        258852
ClmProcedureCode_3        258866
ClmProcedureCode_4        258867
ClmProcedureCode_5        258868
ClmProcedureCode_6        258868
DeductibleAmtPaid              0
ClmAdmitDiagnosisCode     205966
dtype: int64

In [108]:
### Very much sparse data. Drop  proc code 3,4,5,6

In [109]:
to_drop = ['ClmProcedureCode_5','ClmProcedureCode_6']
admit_patient_data = admit_patient_data.drop(to_drop,axis=1)
to_drop = ['ClmProcedureCode_3','ClmProcedureCode_4','ClmProcedureCode_5','ClmProcedureCode_6']
checked_patient_data = checked_patient_data.drop(to_drop,axis=1)

In [110]:
ben_data.dtypes

BeneID                             object
DOB                                object
DOD                                object
Gender                              int64
Race                                int64
RenalDiseaseIndicator              object
State                               int64
County                              int64
NoOfMonths_PartACov                 int64
NoOfMonths_PartBCov                 int64
ChronicCond_Alzheimer               int64
ChronicCond_Heartfailure            int64
ChronicCond_KidneyDisease           int64
ChronicCond_Cancer                  int64
ChronicCond_ObstrPulmonary          int64
ChronicCond_Depression              int64
ChronicCond_Diabetes                int64
ChronicCond_IschemicHeart           int64
ChronicCond_Osteoporasis            int64
ChronicCond_rheumatoidarthritis     int64
ChronicCond_stroke                  int64
IPAnnualReimbursementAmt            int64
IPAnnualDeductibleAmt               int64
OPAnnualReimbursementAmt          

In [111]:
checked_patient_data.dtypes

BeneID                     object
ClaimID                    object
ClaimStartDt               object
ClaimEndDt                 object
Provider                   object
InscClaimAmtReimbursed      int64
AttendingPhysician         object
OperatingPhysician         object
OtherPhysician             object
ClmDiagnosisCode_1         object
ClmDiagnosisCode_2         object
ClmDiagnosisCode_3         object
ClmDiagnosisCode_4         object
ClmDiagnosisCode_5         object
ClmDiagnosisCode_6         object
ClmDiagnosisCode_7         object
ClmDiagnosisCode_8         object
ClmDiagnosisCode_9         object
ClmDiagnosisCode_10        object
ClmProcedureCode_1        float64
ClmProcedureCode_2        float64
DeductibleAmtPaid           int64
ClmAdmitDiagnosisCode      object
dtype: object

In [112]:
for i in range(len(checked_patient_data.columns)):
    print("{} : {}".format(checked_patient_data.columns[i],checked_patient_data[checked_patient_data.columns[i]].duplicated().any()))

BeneID : True
ClaimID : False
ClaimStartDt : True
ClaimEndDt : True
Provider : True
InscClaimAmtReimbursed : True
AttendingPhysician : True
OperatingPhysician : True
OtherPhysician : True
ClmDiagnosisCode_1 : True
ClmDiagnosisCode_2 : True
ClmDiagnosisCode_3 : True
ClmDiagnosisCode_4 : True
ClmDiagnosisCode_5 : True
ClmDiagnosisCode_6 : True
ClmDiagnosisCode_7 : True
ClmDiagnosisCode_8 : True
ClmDiagnosisCode_9 : True
ClmDiagnosisCode_10 : True
ClmProcedureCode_1 : True
ClmProcedureCode_2 : True
DeductibleAmtPaid : True
ClmAdmitDiagnosisCode : True


In [113]:
admit_patient_data.dtypes

BeneID                     object
ClaimID                    object
ClaimStartDt               object
ClaimEndDt                 object
Provider                   object
InscClaimAmtReimbursed      int64
AttendingPhysician         object
OperatingPhysician         object
OtherPhysician             object
AdmissionDt                object
ClmAdmitDiagnosisCode      object
DeductibleAmtPaid         float64
DischargeDt                object
DiagnosisGroupCode         object
ClmDiagnosisCode_1         object
ClmDiagnosisCode_2         object
ClmDiagnosisCode_3         object
ClmDiagnosisCode_4         object
ClmDiagnosisCode_5         object
ClmDiagnosisCode_6         object
ClmDiagnosisCode_7         object
ClmDiagnosisCode_8         object
ClmDiagnosisCode_9         object
ClmDiagnosisCode_10        object
ClmProcedureCode_1        float64
ClmProcedureCode_2        float64
ClmProcedureCode_3        float64
ClmProcedureCode_4        float64
dtype: object

In [141]:
checked_patient_data['ClaimStartDt'] = pd.to_datetime(checked_patient_data['ClaimStartDt'], errors='coerce')
admit_patient_data['ClaimStartDt'] = pd.to_datetime(admit_patient_data['ClaimStartDt'], errors='coerce')

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,DeductibleAmtPaid,ClmAdmitDiagnosisCode
61127,BENE28435,CLM751605,2009-12-30,2009-12-30,PRV51164,30,PHY432396,NaN,NaN,V734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
53885,BENE26307,CLM640390,2009-10-20,2009-10-20,PRV56416,800,PHY342652,NaN,PHY355570,5738,...,4019,25000,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
368736,BENE116615,CLM718712,2009-12-07,2009-12-07,PRV52045,40,PHY424585,NaN,NaN,53020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
403744,BENE126615,CLM569499,2009-09-08,2009-09-08,PRV51929,500,PHY378288,NaN,PHY393147,72081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
106974,BENE41584,CLM397918,2009-06-05,2009-06-05,PRV53918,600,PHY330576,NaN,NaN,9926,...,E8843,71941,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98325,BENE39078,CLM492061,2009-07-27,2009-07-27,PRV56248,600,PHY427881,NaN,NaN,7140,...,78469,185,4293,NaN,NaN,NaN,NaN,NaN,0,NaN
456500,BENE141571,CLM602266,2009-09-27,2009-09-27,PRV56259,3300,PHY388535,NaN,PHY385495,27951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
302241,BENE97692,CLM725701,2009-12-12,2009-12-12,PRV54237,200,PHY432621,NaN,NaN,V725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
437313,BENE136056,CLM317911,2009-04-22,2009-04-22,PRV54317,500,PHY320952,NaN,NaN,V594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [118]:
entities = {
    "Benficiary_Data" : (ben_data,"BeneID"),
    "checked_patient_Data" : (checked_patient_data, "ClaimID"),
    "admitted_patient_Data" : (admit_patient_data,"ClaimID"),
     "labels" : (labels,"Provider")
}
Relationship = {("Benficiary_Data","BeneID","checked_patient_Data","BeneID"),
                ("Benficiary_Data","BeneID","admitted_patient_Data","BeneID"),
                 ("labels","Provider","checked_patient_Data","Provider"),
                ("labels","Provider","admitted_patient_Data","Provider")
               }



In [119]:
for i in range(len(admit_patient_data.columns)):
    print("{} : {}".format(admit_patient_data.columns[i],admit_patient_data[admit_patient_data.columns[i]].duplicated().any()))

BeneID : True
ClaimID : False
ClaimStartDt : True
ClaimEndDt : True
Provider : True
InscClaimAmtReimbursed : True
AttendingPhysician : True
OperatingPhysician : True
OtherPhysician : True
AdmissionDt : True
ClmAdmitDiagnosisCode : True
DeductibleAmtPaid : True
DischargeDt : True
DiagnosisGroupCode : True
ClmDiagnosisCode_1 : True
ClmDiagnosisCode_2 : True
ClmDiagnosisCode_3 : True
ClmDiagnosisCode_4 : True
ClmDiagnosisCode_5 : True
ClmDiagnosisCode_6 : True
ClmDiagnosisCode_7 : True
ClmDiagnosisCode_8 : True
ClmDiagnosisCode_9 : True
ClmDiagnosisCode_10 : True
ClmProcedureCode_1 : True
ClmProcedureCode_2 : True
ClmProcedureCode_3 : True
ClmProcedureCode_4 : True


In [120]:
feature_matrix, feature_df = ft.dfs(
    entities=entities,
    relationships=Relationship,
    target_entity= 'Benficiary_Data',
    verbose=True
                    
)

Built 206 features
Elapsed: 06:34 | Progress: 100%|██████████


In [121]:
feature_matrix

,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,...,NUM_UNIQUE(admitted_patient_Data.MONTH(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(AdmissionDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(ClaimEndDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(ClaimStartDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(AdmissionDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(ClaimEndDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(ClaimStartDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.labels.PotentialFraud)
BeneID,,,,,,,,,,,,,,,,,,,,,
BENE78023,2,1,0,33,650,12,12,2,1,2,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BENE85657,1,2,0,42,30,12,12,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BENE44310,1,1,0,10,260,12,12,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BENE156228,1,1,Y,10,350,12,12,1,2,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BENE119025,1,1,0,52,40,12,12,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BENE124072,2,1,Y,44,40,12,12,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BENE144954,1,1,0,5,430,12,12,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BENE116512,1,1,0,39,770,12,12,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
feature_df

[<Feature: Gender>,
 <Feature: Race>,
 <Feature: RenalDiseaseIndicator>,
 <Feature: State>,
 <Feature: County>,
 <Feature: NoOfMonths_PartACov>,
 <Feature: NoOfMonths_PartBCov>,
 <Feature: ChronicCond_Alzheimer>,
 <Feature: ChronicCond_Heartfailure>,
 <Feature: ChronicCond_KidneyDisease>,
 <Feature: ChronicCond_Cancer>,
 <Feature: ChronicCond_ObstrPulmonary>,
 <Feature: ChronicCond_Depression>,
 <Feature: ChronicCond_Diabetes>,
 <Feature: ChronicCond_IschemicHeart>,
 <Feature: ChronicCond_Osteoporasis>,
 <Feature: ChronicCond_rheumatoidarthritis>,
 <Feature: ChronicCond_stroke>,
 <Feature: IPAnnualReimbursementAmt>,
 <Feature: IPAnnualDeductibleAmt>,
 <Feature: OPAnnualReimbursementAmt>,
 <Feature: OPAnnualDeductibleAmt>,
 <Feature: COUNT(checked_patient_Data)>,
 <Feature: MAX(checked_patient_Data.ClmProcedureCode_1)>,
 <Feature: MAX(checked_patient_Data.ClmProcedureCode_2)>,
 <Feature: MAX(checked_patient_Data.DeductibleAmtPaid)>,
 <Feature: MAX(checked_patient_Data.InscClaimAmtReimbu

In [123]:
feature_matrix['COUNT(admitted_patient_Data)'].max()

5.0

In [124]:
feature_matrix[feature_matrix['COUNT(admitted_patient_Data)'] >4]

,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,...,NUM_UNIQUE(admitted_patient_Data.MONTH(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(AdmissionDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(ClaimEndDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(ClaimStartDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(AdmissionDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(ClaimEndDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(ClaimStartDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.labels.PotentialFraud)
BeneID,,,,,,,,,,,,,,,,,,,,,
BENE120987,1,1,Y,26,350,12,12,2,1,1,...,5.0,3.0,4.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0
BENE64791,2,1,Y,36,440,12,12,1,1,1,...,4.0,4.0,3.0,4.0,3.0,1.0,1.0,1.0,1.0,2.0
BENE119457,2,1,0,52,580,12,12,1,1,1,...,4.0,3.0,4.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0


In [125]:
feature_matrix['COUNT(checked_patient_Data)'].max()

18.0

In [126]:
feature_matrix[feature_matrix['COUNT(checked_patient_Data)']>15]

,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,...,NUM_UNIQUE(admitted_patient_Data.MONTH(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(AdmissionDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(ClaimEndDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(ClaimStartDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(AdmissionDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(ClaimEndDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(ClaimStartDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.labels.PotentialFraud)
BeneID,,,,,,,,,,,,,,,,,,,,,
BENE63544,2,1,Y,5,200,12,12,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BENE118316,2,2,Y,11,750,12,12,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BENE39382,1,1,Y,14,141,12,12,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BENE149242,2,2,Y,14,250,12,12,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
feature_matrix[feature_matrix['NUM_UNIQUE(admitted_patient_Data.labels.PotentialFraud)'] == 1]

,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,...,NUM_UNIQUE(admitted_patient_Data.MONTH(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(AdmissionDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(ClaimEndDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(ClaimStartDt)),NUM_UNIQUE(admitted_patient_Data.WEEKDAY(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(AdmissionDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(ClaimEndDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(ClaimStartDt)),NUM_UNIQUE(admitted_patient_Data.YEAR(DischargeDt)),NUM_UNIQUE(admitted_patient_Data.labels.PotentialFraud)
BeneID,,,,,,,,,,,,,,,,,,,,,
BENE78023,2,1,0,33,650,12,12,2,1,2,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BENE156228,1,1,Y,10,350,12,12,1,2,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BENE104686,2,1,0,33,331,12,12,2,2,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BENE83453,2,1,0,33,520,12,12,1,2,2,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BENE67740,2,2,0,36,170,12,12,2,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BENE115429,2,1,Y,36,250,12,12,2,2,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BENE89815,1,1,0,50,360,12,12,1,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BENE123420,2,1,0,45,600,12,12,2,1,2,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [128]:
feature_matrix_provider, feature_df_provider = ft.dfs(
    entities=entities,
    relationships=Relationship,
    target_entity= 'labels',
    verbose=True                   
)
feature_matrix_provider

Built 429 features
Elapsed: 1:14:09 | Remaining: ? | Progress:   0%|          | Provider: 0/9680 
Elapsed: 01:07 | Progress: 100%|██████████


,PotentialFraud,COUNT(checked_patient_Data),MAX(checked_patient_Data.ClmProcedureCode_1),MAX(checked_patient_Data.ClmProcedureCode_2),MAX(checked_patient_Data.DeductibleAmtPaid),MAX(checked_patient_Data.InscClaimAmtReimbursed),MEAN(checked_patient_Data.ClmProcedureCode_1),MEAN(checked_patient_Data.ClmProcedureCode_2),MEAN(checked_patient_Data.DeductibleAmtPaid),MEAN(checked_patient_Data.InscClaimAmtReimbursed),...,SUM(admitted_patient_Data.Benficiary_Data.County),SUM(admitted_patient_Data.Benficiary_Data.Gender),SUM(admitted_patient_Data.Benficiary_Data.IPAnnualDeductibleAmt),SUM(admitted_patient_Data.Benficiary_Data.IPAnnualReimbursementAmt),SUM(admitted_patient_Data.Benficiary_Data.NoOfMonths_PartACov),SUM(admitted_patient_Data.Benficiary_Data.NoOfMonths_PartBCov),SUM(admitted_patient_Data.Benficiary_Data.OPAnnualDeductibleAmt),SUM(admitted_patient_Data.Benficiary_Data.OPAnnualReimbursementAmt),SUM(admitted_patient_Data.Benficiary_Data.Race),SUM(admitted_patient_Data.Benficiary_Data.State)
Provider,,,,,,,,,,,,,,,,,,,,,
PRV51001,No,7.0,NaN,NaN,0.0,300.0,NaN,NaN,0.000000,110.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRV51003,Yes,32.0,NaN,NaN,70.0,3300.0,NaN,NaN,2.187500,334.687500,...,4970.0,29.0,32840.0,274220.0,204.0,204.0,7890.0,23310.0,21.0,48.0
PRV51004,No,73.0,NaN,NaN,100.0,3300.0,NaN,NaN,3.150685,416.849315,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRV51005,Yes,561.0,NaN,NaN,200.0,4080.0,NaN,NaN,4.046346,255.864528,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRV51007,No,30.0,NaN,NaN,10.0,1600.0,NaN,NaN,0.333333,226.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRV57759,No,18.0,NaN,NaN,40.0,3300.0,NaN,NaN,2.222222,368.888889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRV57760,No,11.0,NaN,NaN,0.0,1100.0,NaN,NaN,0.000000,180.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRV57761,No,39.0,NaN,NaN,90.0,2200.0,NaN,NaN,5.641026,173.076923,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
print(feature_matrix_provider['COUNT(checked_patient_Data)'].min())
print(feature_matrix_provider['COUNT(checked_patient_Data)'].max())


0.0
4217.0


In [130]:
# Large Standard deviation doesn't support claim that a fraudulent provider claims checking a large population
print("Maximum Patient Check Claim By Fraudulent Provider : {}".format(feature_matrix_provider[feature_matrix_provider['PotentialFraud']=='Yes']['COUNT(checked_patient_Data)'].max()))
print("Minimum Patient Check Claim By Fraudulent Provider : {}".format(feature_matrix_provider[feature_matrix_provider['PotentialFraud']=='Yes']['COUNT(checked_patient_Data)'].min()))
print("Mean Patient Check Claim By Fraudulent Provider : {}".format(feature_matrix_provider[feature_matrix_provider['PotentialFraud']=='Yes']['COUNT(checked_patient_Data)'].mean()))
print("Std deviation Patient Check Claim By Fraudulent Provider : {}".format(feature_matrix_provider[feature_matrix_provider['PotentialFraud']=='Yes']['COUNT(checked_patient_Data)'].std()))



Maximum Patient Check Claim By Fraudulent Provider : 4217.0
Minimum Patient Check Claim By Fraudulent Provider : 0.0
Mean Patient Check Claim By Fraudulent Provider : 187.0513833992095
Std deviation Patient Check Claim By Fraudulent Provider : 360.79546512054134


## Now we will take a look at claims with respect to time slices

In [211]:
ben_data = pd.read_csv("./HealthCare/Train_Beneficiarydata.csv").sample(frac=0.5)
admit_patient_data = pd.read_csv("./HealthCare/Train_Inpatientdata.csv").sample(frac=0.5)
checked_patient_data = pd.read_csv("./HealthCare/Train_Outpatientdata.csv").sample(frac=0.5)
checked_patient_data['ClaimStartDt'] = pd.to_datetime(checked_patient_data['ClaimStartDt'], errors='coerce')
admit_patient_data['ClaimStartDt'] = pd.to_datetime(admit_patient_data['ClaimStartDt'], errors='coerce')
labels = pd.read_csv("./HealthCare/Train.csv")
checked = labels.merge(checked_patient_data)
admitted = labels.merge(admit_patient_data)

In [212]:
checked

,Provider,PotentialFraud,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,PRV51001,No,BENE49220,CLM452024,2009-07-04,2009-07-04,70,PHY402512,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,78900
1,PRV51001,No,BENE81544,CLM352646,2009-05-11,2009-05-11,100,PHY392882,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,78605
2,PRV51001,No,BENE61975,CLM260292,2009-03-22,2009-03-22,60,PHY392882,NaN,PHY392882,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,PRV51001,No,BENE134788,CLM546036,2009-08-26,2009-08-26,400,PHY350459,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,PRV51001,No,BENE11727,CLM733300,2009-12-17,2009-12-17,20,PHY383007,NaN,PHY383007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258863,PRV57763,No,BENE134293,CLM392241,2009-06-01,2009-06-01,10,PHY321072,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,V5869
258864,PRV57763,No,BENE86082,CLM673579,2009-11-09,2009-11-09,100,PHY321072,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
258865,PRV57763,No,BENE92755,CLM417569,2009-06-15,2009-06-19,400,PHY321072,NaN,PHY321072,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
258866,PRV57763,No,BENE79721,CLM601898,2009-09-27,2009-09-27,50,PHY321072,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [213]:
## To Get Number of claims By a Provider within Specific Time
checked['count'] = 1

In [214]:
def generate_time_slices(Slice = 'M'):
    gr_Df = checked.groupby('Provider').resample(Slice, on='ClaimStartDt').sum().reset_index()
    return gr_Df.merge(labels)

In [231]:
# Paramters
##  'W'  : weekly frequency
##  'M'  : month end frequency
##  'SM' : semi-month end frequency (15th and end of month)
##  'Q'  : quarter end frequency

sliced = generate_time_slices('M')

### Non Fraud Claims

In [219]:
sliced[sliced['PotentialFraud']== 'No']

,Provider,ClaimStartDt,InscClaimAmtReimbursed,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,count,PotentialFraud
0,PRV51001,2009-01-31,1010,0.0,0.0,0.0,0.0,0.0,0.0,0,2,No
1,PRV51001,2009-02-28,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,No
2,PRV51001,2009-03-31,650,0.0,0.0,0.0,0.0,0.0,0.0,0,3,No
3,PRV51001,2009-04-30,100,0.0,0.0,0.0,0.0,0.0,0.0,0,1,No
4,PRV51001,2009-05-31,800,0.0,0.0,0.0,0.0,0.0,0.0,0,2,No
...,...,...,...,...,...,...,...,...,...,...,...,...
48743,PRV57763,2009-08-31,1010,0.0,0.0,0.0,0.0,0.0,0.0,0,6,No
48744,PRV57763,2009-09-30,5520,0.0,0.0,0.0,0.0,0.0,0.0,40,12,No
48745,PRV57763,2009-10-31,600,0.0,0.0,0.0,0.0,0.0,0.0,0,3,No
48746,PRV57763,2009-11-30,430,0.0,0.0,0.0,0.0,0.0,0.0,0,5,No


In [224]:
print("Maximum Claims By Non Fraudulent in a month :",sliced[sliced['PotentialFraud']== 'No']['count'].max())
print("Minimum Claims By Non Fraudulent in a month :",sliced[sliced['PotentialFraud']== 'No']['count'].min())
print("STD Claims By Non Fraudulent in a month :",sliced[sliced['PotentialFraud']== 'No']['count'].std())
print("Mean Claims By Non Fraudulent in a month :",sliced[sliced['PotentialFraud']== 'No']['count'].mean())

Maximum Claims By Non Fraudulent in a month : 70
Minimum Claims By Non Fraudulent in a month : 0
STD Claims By Non Fraudulent in a month : 6.181734802612447
Mean Claims By Non Fraudulent in a month : 3.7944471392603885


### Fraud Claims

In [222]:
sliced[sliced['PotentialFraud']== 'Yes']

,Provider,ClaimStartDt,InscClaimAmtReimbursed,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,count,PotentialFraud
12,PRV51003,2009-01-31,250,0.0,0.0,0.0,0.0,0.0,0.0,0,5,Yes
13,PRV51003,2009-02-28,1250,0.0,0.0,0.0,0.0,0.0,0.0,0,3,Yes
14,PRV51003,2009-03-31,3500,0.0,0.0,0.0,0.0,0.0,0.0,0,3,Yes
15,PRV51003,2009-04-30,100,0.0,0.0,0.0,0.0,0.0,0.0,70,1,Yes
16,PRV51003,2009-05-31,400,0.0,0.0,0.0,0.0,0.0,0.0,0,2,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
48380,PRV57709,2009-08-31,3340,0.0,0.0,0.0,0.0,0.0,0.0,0,11,Yes
48381,PRV57709,2009-09-30,590,0.0,0.0,0.0,0.0,0.0,0.0,0,5,Yes
48382,PRV57709,2009-10-31,960,0.0,0.0,0.0,0.0,0.0,0.0,0,6,Yes
48383,PRV57709,2009-11-30,860,0.0,0.0,0.0,0.0,0.0,0.0,0,6,Yes


In [225]:
print("Maximum Claims By Non Fraudulent in a month :",sliced[sliced['PotentialFraud']== 'Yes']['count'].max())
print("Minimum Claims By Non Fraudulent in a month :",sliced[sliced['PotentialFraud']== 'Yes']['count'].min())
print("STD Claims By Non Fraudulent in a month :",sliced[sliced['PotentialFraud']== 'Yes']['count'].std())
print("Mean Claims By Non Fraudulent in a month :",sliced[sliced['PotentialFraud']== 'Yes']['count'].mean())

Maximum Claims By Non Fraudulent in a month : 394
Minimum Claims By Non Fraudulent in a month : 0
STD Claims By Non Fraudulent in a month : 31.576664969557328
Mean Claims By Non Fraudulent in a month : 17.34097158570119


#### Above Makes it clear that fraudulent providers made on avg 17 claims of checking Patient
#### Whilst Non Fraudulent on avg checks 3 patient a month

In [228]:
## Analyzing Reimbursed AMount By Fraudulent Provider
print("Maximum Reimbursed Amount By Fraudulent Provider: ",sliced[sliced['PotentialFraud']== 'Yes']['InscClaimAmtReimbursed'].max())
print("Minimum Reimbursed Amount By Fraudulent Provider: ",sliced[sliced['PotentialFraud']== 'Yes']['InscClaimAmtReimbursed'].min())
print("Mean Reimbursed Amount By Fraudulent Provider: ",sliced[sliced['PotentialFraud']== 'Yes']['InscClaimAmtReimbursed'].mean())
print("STD Reimbursed Amount By Fraudulent Provider: ",sliced[sliced['PotentialFraud']== 'Yes']['InscClaimAmtReimbursed'].std())

Maximum Reimbursed Amount By Fraudulent Provider:  117560
Minimum Reimbursed Amount By Fraudulent Provider:  0
Mean Reimbursed Amount By Fraudulent Provider:  5006.6416131989
STD Reimbursed Amount By Fraudulent Provider:  9443.816453667056


In [229]:
## Analyzing Reimbursed AMount By Non Fraudulent Provider
print("Maximum Reimbursed Amount By Fraudulent Provider: ",sliced[sliced['PotentialFraud']== 'No']['InscClaimAmtReimbursed'].max())
print("Minimum Reimbursed Amount By Fraudulent Provider: ",sliced[sliced['PotentialFraud']== 'No']['InscClaimAmtReimbursed'].min())
print("Mean Reimbursed Amount By Fraudulent Provider: ",sliced[sliced['PotentialFraud']== 'No']['InscClaimAmtReimbursed'].mean())
print("STD Reimbursed Amount By Fraudulent Provider: ",sliced[sliced['PotentialFraud']== 'No']['InscClaimAmtReimbursed'].std())

Maximum Reimbursed Amount By Fraudulent Provider:  88910
Minimum Reimbursed Amount By Fraudulent Provider:  0
Mean Reimbursed Amount By Fraudulent Provider:  1087.5859838773013
STD Reimbursed Amount By Fraudulent Provider:  2210.973010722513


In [230]:
### Very clearly large claims means large amount reimbursed